In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA



# Unsupervised methods


In supervised learning, we used known labels to predict unknown labels. In unsupervised learning, we don't have any labeled data to work with, and our primary goal is to identify structures or simplify our data so we can more easily summarize it, visualize it, or use it in a supervised model. In contrast to supervised learning, these methods are often more appropriate for exploratory analyses where you don't necessarily have a goal other than finding out how things "work" in your data. As a result, there's a lot more room for interpretation than you might encounter in other data analysis tasks.

We'll talk about two widely used unsupervised methods that are often used together: **k-means clustering** and  **principal components analysis**. (note: we actually talked about these briefly last week, but we'll explore them in a little more depth here)

## Clustering data with K-means

<p>Clustering methods like k-means are a way of identifying a set of groups with similar characteristics in some data. K-means clustering is one of the simplest and most commonly used methods for clustering. The <b>K</b> in "K-means" is some number of groups chosen by the researcher. </p>
<p>The data set below includes some data on penguins recorded by biologists working in the Palmer archipelago in Antartica. The <code>bill_depth_mm</code> and <code>bill_length_mm</code> are measurements of the penguin's bills. Looking at the scatter plot of these measurements, you can probably eyeball some clear "groups" in the data. Reasonable people might disagree on whether there were 2 groups or 3, or maybe even 4, but there are clusters:



In [ ]:
penguins = pd.read_csv('https://gist.githubusercontent.com/slopp/ce3b90b9168f2f921784de84fa445651/raw/4ecf3041f0ed4913e7c230758733948bc561f434/penguins.csv').dropna()
plt.scatter(x=penguins['bill_length_mm'], y=penguins['bill_depth_mm'])
plt.xlabel('bill length')
plt.ylabel('bill depth')
plt.show()

Instead of just trying to eyeball it, we can use K-means clustering to identify a set of optimal clusters for this set of features. Specifically, K-means attempts to identify the cluster assignments that minimizes the total within-cluster sum of squares. Visually, you can think of it as trying to find clusters where all of the points are as close together as possible.

To use K-Means, we first need to mean-center and standardize our features so they all have mean = 0 and a standard deviation of 1. 

In [ ]:


features = ['bill_length_mm', 'bill_depth_mm']
X = penguins[features] 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Note that scaling doesn't really change anything about the relative positions of any points in our dataset, it just ensures they are both centered on zero:

In [ ]:
plt.scatter(X_scaled[:,0], X_scaled[:,1])
plt.xlabel('scaled bill length')
plt.ylabel('scaled bill depth')
plt.show()

Now we'll apply K-means clustering on the scaled data. We'll initialize a model by calling `KMeans` along with any additional options we want to specifiy. In this example, I'm going to assume there are three clusters, and I'll use the `random_state` variable to ensure reliability. Finally, I'll set the model up to use 10 random initalizations (this is a way to avoid becoming [stuck in a local minimum](https://stats.stackexchange.com/questions/487160/what-does-it-mean-for-the-k-means-algorithm-to-be-trapped-in-a-local-minimum))

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=99, n_init=10)
kmeans.fit(X_scaled)

The `intertia` attribute is the total within cluster sum of squares value that the algorithm is attempting to optimize.

In [ ]:
kmeans.inertia_

The `cluster_centers_` attribute is a K x J array with one row for each cluster and one column for each feature. The values are just the average of the (scaled) features within each cluster.

In [ ]:
kmeans.cluster_centers_

Putting the results in a data frame and adding the feature names can make this a little more readable, and can give you a sense of where each cluster is located in the data. ex: cluster 0 is on the low end of bill length and has moderate depth, while cluster 1 has moderate length and low bill depth. 

In [ ]:
pd.DataFrame(kmeans.cluster_centers_, columns=features)

The `.labels_` attribute of our fitted model has the cluster assignments. This should have a numpy array with one element for each observation in our data, and the numbers should correspond to the value of `K`:

In [ ]:
kmeans.labels_

Right now, these are stored as a numpy array, but we can add them back to our original data frame as a new column to have them in Pandas format, then we can use this new column in visualizations and other analyses.

In [ ]:
penguins['cluster'] = kmeans.labels_


Using a color map to assign colors to each cluster:

In [ ]:
sns.scatterplot(data=penguins, x='bill_length_mm', y='bill_depth_mm', hue='cluster')
plt.show()

Note that, in this instance, the clusters we identified roughly correspond to the different species in the penguins sampled:

In [ ]:
sns.scatterplot(data=penguins, x='bill_length_mm', y='bill_depth_mm', hue='species')
plt.show()

For the purposes of this example, we chose to use just two features and 3 categories, but in most practical scenarios where you would want to using K-means clustering, you'll have a lot more variables you're interested in. Foruntately, the logic here generalizes readily to higher dimensional data. If we want to visualize the results of a cluster analysis that uses more than two variables, we might consider using a dimensionality reduction method like the one we're discussing in the next section so we can get a two-dimensional representation of a higher dimensional data set.


<h3 style="color: red;font-weight: bold;">Question 1: Use <code>K-means</code> clustering on the <code>body_mass_g</code> and <code>flipper_length_mm</code> variables. Use your best judgement to select a value for <code>K</code> and create a scatter plot to visualize your results. </h3>

### Choosing values for K: the elbow method

Picking a number of clusters is often more of an art than a science. There are some wrong choices (it wouldn't make sense to use K=1) but there are usually multiple good values. The elbow method is a hueristic for picking a "good" value for `k` in K-means clustering. It doesn't give us a definitive answer, but it can help guide decision when you're not sure whether there are 4 groups or 5. To use it, we run K-means multiple times with increasing values of K and then we plot the total within cluster sum of squares for each iteration. The "elbow" is the point where the line bends sharply. Since the within cluster sum of squares is our measure of cluster quality, this "elbow" represents the point where we start to see diminishing returns for increasing values of K. 

In [ ]:
# start with an empty list
wcss = []

# test out values from 2 to 20
for k in range(2, 20):
    # get K clusters
    kmeans = KMeans(n_clusters=k, random_state=99, n_init=5)
    kmeans.fit(X_scaled)
    # get the WCSS value (stored in the "interia_" attribute:
    wcss.append(kmeans.inertia_)
    
    

Based on the results, it looks like 3 or 4 clusters is plenty here. Additional values of "K" beyond that don't seem to improve things much. This makes sense given that we're only using two features here - there's just not a lot of complexity, and the clusters we identified already do a good job of explaining the variations in the data.

In [ ]:
# plot the results
fig, ax = plt.subplots(figsize=(12, 3))
plt.plot(range(2, 20), wcss, marker='o')
ax.locator_params(axis='x', nbins=20)
plt.grid()
plt.show()

<div class="alert alert-block alert-info">
<h3>Note</h3>
K-means clustering is one of the most widely used methods for cluster analysis, but there are a variety of alternative models that may perform better for very large data sets, or in cases where clusters are defined by complex non-linear relationships. You can read a little more about some alternative clustering methods supported by scikit-learn <a href='https://scikit-learn.org/stable/modules/clustering.html'>here</a>. Some of these models, such as <a href='https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html'>DBSCAN</a> and <a href = 'https://scikit-learn.org/stable/modules/generated/sklearn.cluster.ward_tree.html'>Ward clustering</a>, will automatically choose a number of clusters based on some characteristics of the data. 
</div>


# PCA

Principal Components Analysis is an unsupervised dimensionality reduction technique that can help simplify large datasets with lots of correlated variables. If we have an `N x K` matrix of correlated variables, PCA will return a new `N x K` matrix with these properties:

1. The columns will be uncorrelated with each other.
2. The columns will be ordered by their importance. In other words, the first column will be some linear combination of the variables that explains the most variation in the data, the second column will explain the second most variation, and so on.

Taken together, these two properties mean that we can often use PCA to take a large number of correlated variables and create 2 or 3 new variables that capture most of the variance in our data. We can then use these results as inputs for a supervised machine learning model, or to do something like visualize a complex data set in a simple scatter plot.



For this section, we'll use some data on European political parties from the 2024 round of the [Chapel Hill Expert Survey](https://www.chesdata.eu/ches-europe) (CHES). The CHES project attempts estimate party positions and ideology by aggregating expert opinions. in essence they might ask 10 experts on UK politics to rank each of the major parties in terms of their support or opposition to increasing immigration, and then they would use the average responses to this question as a measure of Labor's immigration views.


In [ ]:
ches = pd.read_csv("ches_data.csv")
ches.head()

Here are descriptions of the variables used below:

| Variable name  | Variable                                                                                                       | Values                                                                                                                                                                  |
|----------------|----------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| spendvtax      | Position on IMPROVING PUBLIC SERVICES VS. REDUCING TAXES during 2024                                           | 0 = strongly favors improving public services  <br>10 = strongly favors reducing taxes                                                                                  |
| redistribution | Next, where did these political parties stand on REDISTRIBUTION in 2024?                                       | 0 = strongly favors redistribution<br>10 = strongly opposes redistribution                                                                                              |
| climate_change | What was their position towards CLIMATE CHANGE POLICIES during 2024?                                           | 0 = strongly supports climate change policies even at the cost of economic growth<br>10 = strongly supports economic growth even at the cost of climate change policies |
| lgbtq_rights   | Position on policies supporting LGBTQ+ RIGHTS (e.g. marriage equality, adoption rights, transgender rights)    | 0 = strongly supports LGBTQ+ rights<br>10 = strongly opposes LGBTQ+ rights                                                                                              |
| eu_position    | How would you describe the GENERAL POSITION ON EUROPEAN INTEGRATION that the party leadership took during 2024 | 1 = strongly opposed<br>7 = strongly in favor                                                                                                                           |
| protectionism  | Position towards TRADE LIBERALIZATION/ PROTECTIONISM                                                           | 0 = strongly favors trade   liberalization<br>10 = strongly favors protection of domestic producer                                                                      |

We'll start by selecting 4 features of interest:

In [ ]:

features = ['spendvtax', 'redistribution','climate_change', 'lgbtq_rights']
correlation_matrix = ches[features].corr()


We can start by examining a correlation matrix for these features. You'll notice that some of them are strongly correlated with each other. In some cases, correlations are strong enough that we might think that including them together in a model is potentially redundant. If we know a party's position of taxes and spending, we *probably* can make a good guess as to their position on taxes and spending because the two values are so strongly related. One way to think about the usefulness of dimensionality reduction is that it gives us a way to identify and eliminate this kind of redundancy.

In [ ]:
# using seaborn to visualize the correlation matrix as a heatmap
plt.figure(figsize=(4, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

To use PCA, we'll start by scaling the features (notice a trend?)

In [ ]:


X = ches[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


We'll use initialize the PCA model, then use `pca.fit` to fit it to the data. Finally we'll use `transform` to apply the fitted model to the variables in X_scaled. 

In [ ]:
pca = PCA()
prcomp = pca.fit(X_scaled)
compvalues = prcomp.transform(X_scaled)

We'll start by taking a look at the characteristics of the fitted model, then we'll try using `compvalues` for an analysis. 

The `explained_variance_ratio_` attribute of our fitted PCA model is an array that give the proportion of variance explained by each component of the new PCA matrix. 

In [ ]:
prcomp.explained_variance_ratio_

You can read this as saying: "the first component explains about 72% of the variation, the second explains about 23% the third explains 3%... and so on. So, for this set of variables, we can explain over 90% of the variation in our data using just the first two components of this matrix.

We can also use the `prcomp.components_` to see the "loadings" or the importance of each feature in each component. Larger absolute values indicate that a feature is more strongly associated with a particular component, and negative/positive signs indicate that a feature is either positively or negatively associated with that component.

In [ ]:
prcomp.components_

We get this as a numpy array, but we can make it a bit more readable by converting it to a Pandas Dataframe and naming each of the columns. Each row here represents a single component, and each column represents a specific feature. The first dimension is relatively equally associated with all for values, while the second dimnesion is positively associated with `spendvtax` and `redistribution` but negatively associated with `lgbtq_rights` and `climate_change` views.

In [ ]:
components_df = pd.DataFrame(prcomp.components_, columns = features)
components_df.reset_index()

Since the first two principal components explain most of the variation in our data, we should be able to get a reasonable summary of our data by just visualizing observations on those first two component values. We'll extract those components and put them in a data frame, then we'll join this back up with our original data set so we can use the components for more analyses.

In [ ]:
components_frame = pd.DataFrame(compvalues[:,:2], columns =['component 1', 'component 2'])
df = pd.concat([ches, components_frame], axis=1)


To visualize the results, we'll use plotly to create a visualization that shows each party's placement on these two axes. We'll also use a `hovertemplate` to display additional information about each point when hovering. This should allow us to get a good idea of what each component is picking up on:

In [ ]:
import plotly.express as px
fig = px.scatter(df, 
           x='component 1', 
           y='component 2',     
           custom_data =['party','family', 'country', 'spendvtax', 'redistribution', 'lgbtq_rights',
                        'climate_change'],
           width=600,   
           height=500 )
fig.update_traces(hovertemplate=
                  "<b>%{customdata[0]}</b>"
                  '<br>Family: %{customdata[1]}' +
                  "<br>Country: %{customdata[2]}" +
                  "<br>spendvtax: %{customdata[3]:.1f}" +
                  "<br>redistribution: %{customdata[4]:.1f}" +
                  "<br>lgbtq rights: %{customdata[5]:.1f}" +
                  "<br>climate change: %{customdata[6]:.1f}")

<h3 style="color: red;font-weight: bold;">Question 2: Use K-means clustering to identify groups of parties using the features from the previous steps. Create a color-coded version of the scatter plot above, where each point is color coded according to its cluster membership.</h3>



<h3 style="color: red;font-weight: bold;">Question 3: Repeat the above steps, but include <code>protectionism</code> and <code>eu_position</code> as additional features before performing PCA. Discuss the changes in your results and what they might mean.</h3>



### Basic steps for using PCA: 
1. Select a set of numeric features that are correlated (or, if they're categorical, convert them to numeric by dummy-coding)
2. Center and scale the features. 
3. Use your best judgement to select a value for K (you can always change it if your results don't seem right)
4. Perform PCA and examine the results.
5. Choose one or more principal components to use for your analysis. If you're trying to make a visualization, you probably just need the first two, but you could use more if you're using PCA to process features for a supervised model. 

Keep in mind that using less that the total number of available features effectively means you're throwing out some data. But, if a couple of features can already explain most of your variation, you're not losing much information and you gain the benefits of being able to simplify your model.